# Reversal / Momentum - Time Horizon
In this homework, we explore how reversal tends to exist at shorter horizons and momentum at longer horizons. We do this on 4h cryptocurrency price data. Please first run the code below, which will download the price data (from Binance since 01/2020 to 12/2022) and compute returns based on it, stored in `ret`.

1. Use `ret` to generate rank-demeaned-normalized XS reversal strategies at 4,8,12,16,20, and 24 hour frequencies.

In [38]:
from binance.client import Client as bnb_client
from datetime import datetime, timezone
import pandas as pd 
import numpy as np 

client = bnb_client(tld='US')
###  if you're in the US, use: 
### "client = bnb_client(tld='US')" here instead

def get_binance_px(symbol,freq,start_ts = '2022-08-12'):
    data = client.get_historical_klines(symbol,freq,start_ts)
    columns = ['open_time','open','high','low','close','volume','close_time','quote_volume',
    'num_trades','taker_base_volume','taker_quote_volume','ignore']

    data = pd.DataFrame(data,columns = columns)
    
    # Convert from POSIX timestamp (number of millisecond since jan 1, 1970)
    from datetime import datetime, timezone
    data['open_time'] = data['open_time'].map(lambda x: datetime.fromtimestamp(x/1000, tz=timezone.utc))
    data['close_time'] = data['close_time'].map(lambda x: datetime.fromtimestamp(x/1000, tz=timezone.utc))
    return data 

#ret = []
#rev_ret_list = []

univ = ['BTCUSDT','ETHUSDT','ADAUSDT','BNBUSDT','XRPUSDT','DOTUSDT','MATICUSDT']

#freq = ['4h', '8h', '12h', '24h']
f = '4h'  # Example frequency, you can change this to '8h', '12h', or '24h'
px = {}
for x in univ:
    data = get_binance_px(x,f)
    px[x] = data.set_index('open_time')['close']

px = pd.DataFrame(px).astype(float)
px = px.reindex(pd.date_range(px.index[0],px.index[-1],freq=f))
ret = px.pct_change()
ret

C:\Users\mroha\AppData\Local\Temp\ipykernel_13028\282316535.py:11: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  data = client.get_historical_klines(symbol,freq,start_ts)
C:\Users\mroha\AppData\Local\Temp\ipykernel_13028\282316535.py:37: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret = px.pct_change()


,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2022-08-12 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-12 04:00:00+00:00,-0.002232,-0.007980,0.001109,-0.000824,NaN,-0.006486,-0.009698
2022-08-12 08:00:00+00:00,-0.012102,-0.010549,-0.018095,-0.011183,NaN,-0.009793,-0.009793
2022-08-12 12:00:00+00:00,0.013096,0.016003,0.015331,0.012069,NaN,0.032967,0.010989
2022-08-12 16:00:00+00:00,0.008303,0.015435,0.012709,0.004365,NaN,0.001064,0.007609
...,...,...,...,...,...,...,...
2025-08-13 04:00:00+00:00,-0.000509,-0.004508,0.002801,0.010769,-0.000308,0.002893,0.000000
2025-08-13 08:00:00+00:00,0.009365,0.013323,0.024328,0.010998,0.014316,0.024038,0.000000
2025-08-13 12:00:00+00:00,0.004163,-0.005471,-0.007273,-0.011464,-0.010251,-0.015962,0.000000
2025-08-13 16:00:00+00:00,0.008637,0.015937,0.011905,0.002049,0.007315,0.007872,0.000000


2. IE. for the 12 hour strategy, you will use the average return in the last three 4-hour bars to form a rank-demeaned-noramlzied XS portfolio and hold for the 4 hours.


In [73]:
strats = {}
for hor in [1, 2, 3, 4, 5, 6]:
    lookback_ret = ret.rolling(window=hor, min_periods=1).mean()  # Average return over the last 'hor' periods
    rank_ret = lookback_ret.rank(axis=1)
    demean_ret = rank_ret.sub(rank_ret.mean(axis=1), axis=0)
    norm_demean_ret = demean_ret.div(demean_ret.abs().sum(axis=1), axis=0)
    weights = norm_demean_ret.shift(1)  # Shift to avoid lookahead bias
    portfolios = ret.mul(weights, axis=1)
    strats[hor] = portfolios.sum(axis=1)
strats = pd.DataFrame(strats)
strats

,1,2,3,4,5,6
2022-08-12 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-12 04:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-12 08:00:00+00:00,-0.002540,-0.002540,-0.002540,-0.002540,-0.002540,-0.002540
2022-08-12 12:00:00+00:00,0.003545,0.000795,0.000795,0.000795,0.000795,0.000795
2022-08-12 16:00:00+00:00,0.000272,-0.001428,-0.003908,-0.003908,-0.003908,-0.003908
...,...,...,...,...,...,...
2025-08-13 04:00:00+00:00,0.000352,-0.000125,0.000957,0.000301,0.000301,0.000301
2025-08-13 08:00:00+00:00,0.002699,0.003885,0.000048,0.005685,0.008445,0.007552
2025-08-13 12:00:00+00:00,-0.005071,-0.004869,-0.005273,-0.002174,-0.003525,-0.004774
2025-08-13 16:00:00+00:00,0.000568,0.003950,0.001806,0.003740,0.003654,0.003864


3. Compute the Sharpe ratios of the reversal strategy at these different horizons.

In [68]:
sr = strats.mean()/strats.std()*np.sqrt(252*24/4)
sr

1   -2.082476
2   -0.840429
3   -0.683377
4   -0.081829
5   -0.197055
6    0.255085
dtype: float64

4. At what horizons do you observe reversal vs. momentum?

Reversal is most prominent in the 4-8 hr time horizon and sharpe becomes worse as time goes up.

5. The first bar typically contains the most reversal. "Skip" the first bar by lagging your portfolio by one 4h bar (similar to how UMD does at the monthly frequency). This should strengthen any momentum you see.

In [70]:
strats_lag = {}
for hor in [1, 2, 3, 4, 5, 6]:
    lookback_ret = ret.rolling(window=hor).mean()  # Average return over the last 'hor' periods
    rank_ret = lookback_ret.rank(axis=1, method='first', ascending=True)
    demean_ret = rank_ret.sub(rank_ret.mean(axis=1), axis=0)
    norm_demean_ret = demean_ret.div(demean_ret.abs().sum(axis=1), axis=0)
    weights = norm_demean_ret.shift(2)  # Shift to avoid lookahead bias
    portfolios = ret.mul(weights, axis=1)
    strats_lag[hor] = portfolios.sum(axis=1)
strats_lag = pd.DataFrame(strats)
strats_lag

,1,2,3,4,5,6
2022-08-12 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-12 04:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-12 08:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-12 12:00:00+00:00,-0.000553,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-12 16:00:00+00:00,-0.002008,-0.002737,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
2025-08-13 04:00:00+00:00,-0.001479,0.000690,0.000917,-0.000006,-0.000623,-0.000623
2025-08-13 08:00:00+00:00,0.005166,-0.000064,0.006465,0.007552,0.007552,0.007552
2025-08-13 12:00:00+00:00,-0.004604,-0.002316,-0.000685,-0.003872,-0.004774,-0.003900
2025-08-13 16:00:00+00:00,0.003287,0.001083,0.002410,0.004200,0.003754,0.003567


In [71]:
sr = strats_lag.mean()/strats_lag.std()*np.sqrt(252*24/4)
sr

1    0.595738
2    0.520848
3    1.410435
4    0.454653
5    0.898373
6    1.157244
dtype: float64